# 악성 URL 분류 AI 모델 요약 및 서비스 구축

이 노트북은 데이터 전처리, AI 모델 학습, 성능 평가, 모델 저장, Flask를 활용한 로컬 예측 서비스까지 전체 과정을 담고 있습니다. 각 단계별로 왜 해당 방법이 합당한지 설명도 포함되어 있습니다.

## 1. 데이터 준비 및 전처리
- 이미 전처리된 데이터(malicious_phish_data_step3.csv)를 사용합니다.
- 다양한 Feature(특징)를 활용하여 악성/정상 URL을 분류합니다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(r'C:\Users\kim08\Desktop\GSM_BIGDATA\GSM_BIGDATA\malicious_phish_data_step3.csv')
X = df.drop(columns=['type'])
y = df['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

Train shape: (7396, 14) Test shape: (1849, 14)


## 2. AI 모델 학습 및 평가
- 여러 모델 중 XGBoost를 최종 선택했습니다.
- 이유: Random Forest, Decision Tree 등과 비교해 성능(정확도, F1 Score)이 가장 높았고, 대용량 데이터와 복잡한 Feature 조합에서 강점을 보였습니다.

In [2]:
!pip install xgboost

In [3]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=77,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

print('정확도:', accuracy_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred))

c:\Users\kim08\anaconda3\envs\myenv\lib\site-packages\xgboost\training.py:183: UserWarning: [01:19:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


정확도: 0.9626825310978907
F1 Score: 0.9620643658800787
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1370
           1       0.97      0.88      0.92       479

    accuracy                           0.96      1849
   macro avg       0.97      0.94      0.95      1849
weighted avg       0.96      0.96      0.96      1849



### 모델 선택 근거
- Decision Tree: 해석은 쉽지만 예측력이 낮음
- Random Forest: 과적합에 강하고 안정적이지만, XGBoost가 더 높은 성능을 보임
- XGBoost: 실제 실험에서 가장 높은 정확도와 F1 Score를 보여 최종 선택

## 3. 모델 저장 (pkl 파일)
- 학습된 모델을 joblib을 이용해 저장합니다.

In [4]:
import joblib
joblib.dump(xgb_model, 'xgb_model.pkl')
print('모델이 xgb_model.pkl로 저장되었습니다.')

모델이 xgb_model.pkl로 저장되었습니다.


## 4. Flask를 활용한 로컬 예측 서비스
- 저장된 모델(pkl 파일)을 불러와서 REST API 형태로 예측 서비스를 제공합니다.
- 아래 코드를 app.py로 저장 후, 같은 폴더에서 실행하면 됩니다.

In [ ]:

flask_code = """from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)
model = joblib.load('xgb_model.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    features = np.array([list(data.values())])
    prediction = model.predict(features)[0]
    return jsonify({'prediction': int(prediction)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
"""

with open('app.py', 'w') as f:
    f.write(flask_code)
print("Flask 예측 서버 파일(app.py)이 생성되었습니다.")

### 예측 API 사용 예시 (Python)
```python
import requests
url = 'http://localhost:5000/predict'
data = {
    'len_url': 58,
    'len_sub_domain': 12,
    'len_root_domain': 8,
    'len_suffix': 3,
    'is_private': 0,
    'count_special_char': 5,
    'num_port': 0,
    'len_query': 0,
    'count_sub_domain': 2,
    'is_filter': 0,
    'ratio_alpha_numeric': 0.85,
    'count_url_dots': 3,
    'value_entropy_url': 3.5,
    'count_file_path': 0
}
response = requests.post(url, json=data)
print(response.json())
```